In [60]:
import numpy as np
import tensorflow as tf

from typing import List, Container
from tensorflow.data import Dataset
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Concatenate, Dense, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import Mean

class TextCNN(Model):
    def __init__(self, class_num: int):
        super(TextCNN, self).__init__()
        self.class_num: int = class_num
        
    def build(self, input_shape):
        self.conv_1: Conv1D = Conv1D(filters=128, kernel_size=1, activation="relu", name="conv_1")
        self.pool_1: MaxPool1D = MaxPool1D(pool_size=2, strides=1,name="pool_1")
        self.conv_2: Conv1D = Conv1D(filters=128, kernel_size=2, activation="relu", name="conv_2")
        self.pool_2: MaxPool1D = MaxPool1D(pool_size=2, strides=1, name="pool_2")
        self.concatenate: Concatenate = Concatenate(axis=1)
        self.flatten: Flatten = Flatten()
        self.dense: Dense = Dense(self.class_num, activation="softmax")
        super(TextCNN, self).build(input_shape)

    def call(self, inputs: Dataset, training=None, mask=None):
        x = self.conv_1(inputs)
        x = self.pool_1(x)
        convs: List[Conv1D] = [self.conv_1(inputs), self.conv_2(inputs)]
        pools: List[MaxPool1D] = [self.pool_1(convs[0]), self.pool_2(convs[1])]
        x = self.concatenate(pools)
        x = self.flatten(x)
        x = self.dense(x)
        return x
    
    def summary(self):
        input: Input = Input(shape=(3, 100), name="Input")
        output = self.call(input)
        model = Model(inputs=input, outputs=output, name="TextCNN")
        model.summary()


In [3]:
from user_profile_prediction.etl.preprocess_train_data import PreprocessTrainingData
p = PreprocessTrainingData("/Volumes/Samsung_T5/Files/Document/小象学院/GroupProject/project_data/data/train.csv")
model = p.train_word2vec_model()

ts = tf.data.Dataset.from_generator(p.age_data_iter, (tf.float32, tf.int8))

/Users/luominzhi/Code/Python/user_profile_prediction/user_profile_prediction/etl/preprocess_train_data.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df: DataFrame = pd.read_csv(file_path, sep="###__###", header=None)
1000it [00:51, 19.35it/s]


In [49]:
train_x, train_y = [], []

for x, y in p.age_data_iter():
    train_x.append(x)
    train_y.append(y)

train_x, train_y = np.array(train_x).astype(np.float32), np.array(train_y)
train_y = tf.one_hot(train_y, depth=tf.unique(train_y).y.shape[0])

In [64]:
text_cnn = TextCNN(7)

optimizer: Adam = Adam(learning_rate=1e-3)
losses: CategoricalCrossentropy = CategoricalCrossentropy()

train_loss = Mean(name="TrainLoss")

In [65]:
text_cnn.build(input_shape=(None, 3, 100))
text_cnn.summary()



Model: "TextCNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 3, 100)]     0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 3, 128)       12928       Input[0][0]                      
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 2, 128)       25728       Input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling1D)           (None, 2, 128)       0           conv_1[1][0]                     
____________________________________________________________________________________________

In [66]:
text_cnn(train_x[:2])

<tf.Tensor: shape=(2, 7), dtype=float32, numpy=
array([[0.1445455 , 0.15985623, 0.14281403, 0.13101424, 0.15293336,
        0.13575216, 0.1330845 ],
       [0.18937816, 0.21390799, 0.08726924, 0.17753533, 0.12916042,
        0.07897791, 0.12377098]], dtype=float32)>

In [67]:
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        prediction = model(features)
        loss = losses(labels, prediction)
    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    
    train_loss.update_state(loss)
    
    return gradient

In [69]:
g = train_step(text_cnn, train_x[:10], train_y[:10])


In [73]:
g


[<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
 array([[[ 2.3434185e-03,  3.3510486e-03, -3.7469687e-03, ...,
          -1.2890428e-03, -2.2747477e-03, -3.1755251e-04],
         [-2.5557401e-02, -2.4730591e-02,  3.6886036e-03, ...,
           3.3400450e-03, -3.4728937e-03, -1.0641187e-03],
         [ 8.2698390e-03,  5.2105002e-03, -1.3956956e-03, ...,
          -2.2191519e-03,  1.1492990e-03,  7.0764287e-04],
         ...,
         [-1.3922924e-02, -4.8102015e-03,  4.7201659e-03, ...,
           1.3217740e-03, -1.3103535e-03, -2.7943327e-04],
         [ 2.0711005e-02,  1.8972368e-03, -4.9397717e-03, ...,
          -9.0501882e-05, -3.2650458e-03,  5.9218175e-04],
         [-4.4813692e-03, -1.0442434e-02, -2.0458207e-03, ...,
          -1.1104675e-03,  8.3952653e-04,  7.7296290e-06]]], dtype=float32)>,
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-7.38137588e-02, -7.17725381e-02,  1.77699737e-02,  2.22037397e-02,
         5.89831453e-03, -1.41803948e-02, -9.42107514e

In [75]:
text_cnn.compile(optimizer=optimizer, loss=losses)
text_cnn.fit(train_x, train_y, epochs=5, batch_size=100)


Epoch 1/5
2955/2955 [==============================] - 4s 1ms/step - loss: 1.4354